In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
block_size = 8
batch_size = 4
max_iters = 50000
eval_interval = 2500
learning_rate = 0.01
eval_iters = 250

cuda


In [2]:
with open('Hamlet.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '$', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '£', '·', 'à', 'â', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'ô', 'ö', 'œ', '—', '‘', '’', '“', '”', '\ufeff']


In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data)

tensor([101,  44,  32,  ...,  55,  68,  10])


In [4]:
n = int(0.8*len(data))
train_data, val_data = data[:n], data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x, y = get_batch("train")
print('inputs: ')

print(x)
print('targets: ')
print(y)

    

inputs: 
tensor([[55, 65, 63, 68, 61,  1, 75, 70],
        [72, 59,  8,  1, 74, 62, 63, 68],
        [ 1, 73, 59, 55,  1, 58, 59, 67],
        [ 1, 57, 55, 66, 67,  1, 75, 68]], device='cuda:0')
targets: 
tensor([[65, 63, 68, 61,  1, 75, 70,  1],
        [59,  8,  1, 74, 62, 63, 68, 65],
        [73, 59, 55,  1, 58, 59, 67, 55],
        [57, 55, 66, 67,  1, 75, 68, 57]], device='cuda:0')


In [5]:
x = train_data[:block_size]
y = train_data[1:1+block_size]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context}, target is {target}')

When input is tensor([101]), target is 44
When input is tensor([101,  44]), target is 32
When input is tensor([101,  44,  32]), target is 29
When input is tensor([101,  44,  32,  29]), target is 1
When input is tensor([101,  44,  32,  29,   1]), target is 44
When input is tensor([101,  44,  32,  29,   1,  44]), target is 42
When input is tensor([101,  44,  32,  29,   1,  44,  42]), target is 25
When input is tensor([101,  44,  32,  29,   1,  44,  42,  25]), target is 31


In [6]:
@torch.no_grad()

def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logit, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train

    return out


In [7]:
class BigramLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self,index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        
        return index
    
model = BigramLM(vocab_size)
m = model.to(device)

In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"Iter: {iter}, train loss: {losses['train']}")

    xb, yb = get_batch("train")

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

Iter: 0, train loss: 5.125884532928467
Iter: 250, train loss: 3.528862714767456
Iter: 500, train loss: 2.9139833450317383
Iter: 750, train loss: 2.7183170318603516
Iter: 1000, train loss: 2.6080288887023926
Iter: 1250, train loss: 2.5917985439300537
Iter: 1500, train loss: 2.549973249435425
Iter: 1750, train loss: 2.5079760551452637
Iter: 2000, train loss: 2.5331900119781494
Iter: 2250, train loss: 2.529125452041626
Iter: 2500, train loss: 2.5195679664611816
Iter: 2750, train loss: 2.5523593425750732
Iter: 3000, train loss: 2.53202748298645
Iter: 3250, train loss: 2.4845242500305176
Iter: 3500, train loss: 2.4990310668945312
Iter: 3750, train loss: 2.52143931388855
Iter: 4000, train loss: 2.501692533493042
Iter: 4250, train loss: 2.482009172439575
Iter: 4500, train loss: 2.504254102706909
Iter: 4750, train loss: 2.506941556930542
Iter: 5000, train loss: 2.5180046558380127
Iter: 5250, train loss: 2.4894299507141113
Iter: 5500, train loss: 2.4909679889678955
Iter: 5750, train loss: 2.506

In [9]:
context = torch.zeros((1,1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



“I aniosuand vetr har.
Suend tiro in. won.”—is ns,
Mrd lorelyo ouce athap, be o thes aly, le t the it anldessacorery m nt, shas alll nvongin te angle? n ospofe hed s.
bevetidofoulan d She at t.” Long in d, foretougl gheamor’d
owr oowire wil-ppha wariselack.

Mive
Shers  slly

bior s,
as, Lothetthiof s tilal oureased glale wen e?”

bly. yme reesee
thag mie hooure, acttomom d ritor e s e Se?
ft idutokn dof, amet of; inos wa t
waro
’s ifentinineanecolfoio bom
hivesperuthue?”
Octh! ot thth-whende g
